# EXPLORING THE DUCKDB DATABASE

In [1]:
import duckdb

duckdb_path = "data/duckdb/mobility_analysis.duckdb"
connection = duckdb.connect(duckdb_path)

tables = connection.execute("SHOW TABLES").fetchall()
# Print the tables
print("Tables in the database:", tables)


Tables in the database: [('CONSOLIDATE_CITY',), ('CONSOLIDATE_STATION',), ('CONSOLIDATE_STATION_STATEMENT',), ('DIM_CITY',), ('DIM_STATION',), ('FACT_STATION_STATEMENT',)]


In [2]:
data = connection.execute("SELECT * FROM CONSOLIDATE_STATION_STATEMENT").fetchdf()
data

,STATION_ID,BICYCLE_DOCKS_AVAILABLE,BICYCLE_AVAILABLE,LAST_STATEMENT_DATE,CREATED_DATE
0,1-16107,33,2,2024-11-29,2024-11-29
1,1-44015,9,11,2024-11-29,2024-11-29
2,1-14014,53,4,2024-11-29,2024-11-29
3,1-9020,17,3,2024-11-29,2024-11-29
4,1-11104,20,0,2024-11-29,2024-11-29
...,...,...,...,...,...
1680,3-34,20,2,2024-11-29,2024-11-29
1681,3-240,15,3,2024-11-29,2024-11-29
1682,3-275,12,2,2024-11-29,2024-11-29
1683,3-18,1,20,2024-11-29,2024-11-29


# VALIDATION

In [3]:
import duckdb

duckdb_path = "data/duckdb/mobility_analysis.duckdb"
connection = duckdb.connect(duckdb_path)


sql1 = """SELECT dm.NAME, tmp.SUM_BICYCLE_DOCKS_AVAILABLE
FROM DIM_CITY dm INNER JOIN (
    SELECT CITY_ID, SUM(BICYCLE_DOCKS_AVAILABLE) AS SUM_BICYCLE_DOCKS_AVAILABLE
    FROM FACT_STATION_STATEMENT
    WHERE CREATED_DATE = (SELECT MAX(CREATED_DATE) FROM CONSOLIDATE_STATION)
    GROUP BY CITY_ID
) tmp ON dm.ID = tmp.CITY_ID
WHERE lower(dm.NAME) in ('paris', 'nantes', 'vincennes', 'toulouse');"""

data = connection.execute(sql1).fetchall()
data




[('Toulouse', 1254), ('Nantes', 1210), ('Paris', 18544), ('Vincennes', 214)]

In [4]:
sql2 = """SELECT ds.name, ds.code, ds.address, tmp.avg_dock_available
FROM DIM_STATION ds JOIN (
    SELECT station_id, AVG(BICYCLE_AVAILABLE) AS avg_dock_available
    FROM FACT_STATION_STATEMENT
    GROUP BY station_id
) AS tmp ON ds.id = tmp.station_id;"""

data = connection.execute(sql1).fetchall()
data

[('Toulouse', 1254), ('Nantes', 1210), ('Paris', 18544), ('Vincennes', 214)]